<a href="https://colab.research.google.com/github/dbh25-10120-cloud/DynamicBind/blob/main/DynamicBind_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Check GPU and Setup

In [ ]:
import torch
import os
import shutil
import subprocess

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("WARNING: No GPU detected. Go to Runtime > Change runtime type and select GPU.")

# Set working directory
os.chdir('/content')

# If a non-git `DynamicBind` folder exists, remove it so a fresh clone can proceed
if os.path.exists('DynamicBind') and not os.path.exists(os.path.join('DynamicBind', '.git')):
    print("Removing non-git 'DynamicBind' directory to allow fresh clone...")
    shutil.rmtree('DynamicBind')

# Clone from original repo (or you can upload a ZIP)
if not os.path.exists('DynamicBind'):
    print("Cloning DynamicBind repository...")
    subprocess.run(['git', 'clone', 'https://github.com/luwei0917/DynamicBind.git', 'DynamicBind'], check=True)
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists (git repo)")

os.chdir('/content/DynamicBind')
print(f"Working directory: {os.getcwd()}")


CUDA available: True
GPU: Tesla T4
GPU Memory: 15.83 GB
✓ Repository already exists (git repo)
Working directory: /content/DynamicBind


## Step 2: Install Dependencies

In [ ]:
print("Installing PyTorch...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("Installing PyTorch Geometric...")
!pip install -q torch-geometric

print("Installing RDKit and other dependencies...")
!pip install -q rdkit biopython pandas numpy scipy scikit-learn matplotlib tqdm pyyaml requests

print("Installing ESM and spyrmsd...")
!pip install -q fair-esm spyrmsd e3nn

print("\n✓ All dependencies installed successfully")

Installing PyTorch...
Installing PyTorch Geometric...
Installing RDKit and other dependencies...
Installing ESM and spyrmsd...

✓ All dependencies installed successfully


In [ ]:
import subprocess
import os

# Detect if running in Colab
try:
    import google.colab
    in_colab = True
except ImportError:
    in_colab = False

if in_colab:
    print("✓ Running in Colab - using notebook kernel (conda not available)")
    print("  All required packages were installed in Step 2 via pip")
    print("  Skipping conda environment setup")
else:
    # Local machine with conda installed
    os.chdir('/content/DynamicBind')

    print("Setting up Conda environment for DynamicBind...")
    print("This step ensures all PyTorch Geometric dependencies are installed.\n")

    # Create conda environment from environment-minimal.yml
    if not os.path.exists(os.path.expanduser('~/miniconda3/envs/dynamicbind-minimal')):
        print("Creating dynamicbind-minimal environment...")
        result = subprocess.run(['conda', 'env', 'create', '-f', 'environment-minimal.yml', '-y'],
                              capture_output=True, text=True)
        if result.returncode == 0:
            print("✓ Environment created successfully")
        else:
            print("⚠ Environment creation had issues:")
            print(result.stderr[-500:])  # Last 500 chars
    else:
        print("✓ Environment already exists")

    # Verify torch_cluster is installed
    print("\nVerifying torch_cluster installation...")
    result = subprocess.run(['conda', 'run', '-n', 'dynamicbind-minimal', 'python', '-c',
                            'import torch_cluster; print("✓ torch_cluster found")'],
                           capture_output=True, text=True)
    if result.returncode == 0:
        print(result.stdout.strip())
    else:
        print("⚠ torch_cluster not found, installing...")
        subprocess.run(['conda', 'install', '-n', 'dynamicbind-minimal',
                       '-c', 'pytorch', 'pytorch-cluster', '-y'],
                      capture_output=True)
        print("✓ torch_cluster installed")

    print("\n✓ Conda environment setup complete!")


✓ Running in Colab - using notebook kernel (conda not available)
  All required packages were installed in Step 2 via pip
  Skipping conda environment setup


## Step 3: Clone/Setup DynamicBind Repository

In [ ]:
import subprocess
import os

os.chdir('/content')

# Clone from original repo (or you can upload a ZIP)
if not os.path.exists('DynamicBind'):
    !git clone https://github.com/luwei0917/DynamicBind.git
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

os.chdir('/content/DynamicBind')
print(f"Working directory: {os.getcwd()}")

✓ Repository already exists
Working directory: /content/DynamicBind


## Step 4: Download Model Checkpoints from Zenodo

In [ ]:
import os
import subprocess

os.chdir('/content/DynamicBind')
os.makedirs('workdir', exist_ok=True)

# Download checkpoint from Zenodo (v2)
checkpoint_url = 'https://zenodo.org/records/10183369/files/workdir.zip'
checkpoint_file = 'workdir.zip'

if not os.path.exists(os.path.join('workdir', 'big_score_model_sanyueqi_with_time')):
    print(f"Downloading model checkpoint from Zenodo (may take 5-10 minutes)...")
    !wget -O {checkpoint_file} {checkpoint_url}
    print("Extracting...")
    !unzip -o {checkpoint_file} -d workdir
    # Fix nested directory structure if needed
    if os.path.exists('workdir/workdir'):
        !mv workdir/workdir/* workdir/
        !rmdir workdir/workdir
    print("✓ Checkpoints ready")
    !rm {checkpoint_file}
else:
    print("✓ Checkpoints already available")

print(f"Workdir contents: {os.listdir('workdir')}")

✓ Checkpoints already available
Workdir contents: ['workdir', 'big_score_model_sanyueqi_with_time']


## Step 5: Upload Input Files (PDB & SDF)

In [ ]:
from google.colab import files
import os

os.makedirs('/content/DynamicBind/input', exist_ok=True)
os.chdir('/content/DynamicBind')

print("Upload your protein PDB file and ligand SDF file.")
print("Click 'Choose Files' and select both files.\n")

uploaded = files.upload()

pdb_file = None
sdf_file = None

for fname in uploaded.keys():
    if fname.endswith('.pdb'):
        pdb_file = f'input/{fname}'
        os.rename(fname, pdb_file)
    elif fname.endswith('.sdf'):
        sdf_file = f'input/{fname}'
        os.rename(fname, sdf_file)

if pdb_file and sdf_file:
    print(f"✓ PDB: {pdb_file}")
    print(f"✓ SDF: {sdf_file}")
else:
    print("ERROR: Please upload both PDB and SDF files.")
    if pdb_file:
        print(f"  PDB found: {pdb_file}")
    if sdf_file:
        print(f"  SDF found: {sdf_file}")

Upload your protein PDB file and ligand SDF file.
Click 'Choose Files' and select both files.



Saving Conformer3D_COMPOUND_CID_977.sdf to Conformer3D_COMPOUND_CID_977.sdf
Saving unnamed.pdb to unnamed.pdb
✓ PDB: input/unnamed.pdb
✓ SDF: input/Conformer3D_COMPOUND_CID_977.sdf


## Step 6: Prepare Input CSV

In [ ]:
import pandas as pd
import os

os.chdir('/content/DynamicBind')

# Create input CSV
pdb = os.path.abspath('input/unnamed.pdb') if os.path.exists('input/unnamed.pdb') else None
sdf = os.path.abspath('input/Conformer3D_COMPOUND_CID_977.sdf') if os.path.exists('input/Conformer3D_COMPOUND_CID_977.sdf') else None

# Auto-detect uploaded files
for f in os.listdir('input'):
    if f.endswith('.pdb'):
        pdb = os.path.abspath(f'input/{f}')
    elif f.endswith('.sdf'):
        sdf = os.path.abspath(f'input/{f}')

if pdb and sdf:
    df = pd.DataFrame({'ligand': [sdf], 'protein_path': [pdb]})
    df.to_csv('data/input_compounds.csv', index=False)
    print(f"✓ Input CSV created")
    print(df)
else:
    print(f"ERROR: Missing files. PDB={pdb}, SDF={sdf}")

✓ Input CSV created
                                              ligand  \
0  /content/DynamicBind/input/Conformer3D_COMPOUN...   

                             protein_path  
0  /content/DynamicBind/input/unnamed.pdb  


## Step 7: Run DynamicBind Inference

In [ ]:
!pip uninstall -y torch torchvision torchaudio torch-geometric \
  torch-cluster torch-scatter torch-sparse torch-spline-conv

In [ ]:
!pip install -q torch==2.2.0 torchvision torchaudio \
  --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.2.0+cu118
11.8


In [ ]:
!pip install --no-cache-dir torch-scatter torch-sparse torch-cluster torch-spline-conv \
  -f https://data.pyg.org/whl/torch-2.2.0+cu118.html

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu118.html


In [ ]:
!pip install -q torch-geometric

In [ ]:
from IPython.display import display, HTML

display(HTML("""
<div style="font-family: 'Courier New'; font-size:40px; color:#1f77b4;">
런타임 재시작 필요
</div>
"""))

In [ ]:
!pip uninstall -y numpy
!pip install -q "numpy<2"

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible

In [ ]:
from IPython.display import display, HTML

display(HTML("""
<div style="font-family: 'Courier New'; font-size:40px; color:#1f77b4;">
런타임 재시작 필요
</div>
"""))

In [ ]:
import numpy as np
print(np.__version__)

1.26.4


In [ ]:
import numpy as np
import torch
import torch_cluster
import torch_geometric

print("NumPy:", np.__version__)
print("Torch:", torch.__version__)
print("All OK")

NumPy: 1.26.4
Torch: 2.2.0+cu118
All OK


In [ ]:
import os

# Detect if running in Colab
try:
    import google.colab
    in_colab = True
except ImportError:
    in_colab = False

os.chdir('/content/DynamicBind')

print("Running DynamicBind inference...\n")

if in_colab:
    # Colab: run directly in notebook kernel (all packages installed via pip in Step 2)
    print("(Running in Colab kernel environment)\n")
    !python run_single_protein_inference.py \
      data/input_compounds.csv data/input_compounds.csv \
      --protein_path_in_ligandFile \
      --no_clean \
      --ligand_is_sdf \
      --no_relax \
      --samples_per_complex 10 \
      --savings_per_complex 10 \
      --inference_steps 20 \
      --header colab_run \
      --python /usr/bin/python3 \
      --relax_python /usr/bin/python3 \
      --device 0
else:
    # Local machine: use conda environment for isolation
    print("(Running with Conda environment 'dynamicbind-minimal')\n")
    !conda run -n dynamicbind-minimal python run_single_protein_inference.py \
      data/input_compounds.csv data/input_compounds.csv \
      --protein_path_in_ligandFile \
      --no_clean \
      --ligand_is_sdf \
      --no_relax \
      --samples_per_complex 10 \
      --savings_per_complex 10 \
      --inference_steps 20 \
      --header colab_run \
      --python /usr/bin/python3 \
      --relax_python /usr/bin/python3 \
      --device 0

print("\n✓ Inference step complete!")


Running DynamicBind inference...

(Running in Colab kernel environment)

INFO:root:run_single_protein_inference.py data/input_compounds.csv data/input_compounds.csv --protein_path_in_ligandFile --no_clean --ligand_is_sdf --no_relax --samples_per_complex 10 --savings_per_complex 10 --inference_steps 20 --header colab_run --python /usr/bin/python3 --relax_python /usr/bin/python3 --device 0
2025_12_24_08_53
--------------------------------

/content/DynamicBind/run_single_protein_inference.py /content/DynamicBind
100% 1/1 [00:00<00:00, 76.55it/s]
hub dir /content/DynamicBind/esm_models
Transferred model to GPU
Read data/prepared_for_esm_colab_run.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
100% 201/201 [01:06<00:00,  3.04it/s]
100% 201/201 [01:29<00:00,  2.24it/s]
Reading molecules and generating local structures with RDKit
1it [00:00, 11.72it/s]
Reading language model embeddings.
Generating graphs for ligands and proteins
loading complexes: 100% 1/1 [00:00<00:00,  5.55

In [ ]:
import os
import subprocess

os.chdir('/content/DynamicBind')

print("=== Diagnostic Check ===\n")

print("1. Checking Conda environment...")

try:
    import google.colab
    print("✓ Running in Colab - conda not available (expected)")
except ImportError:
    result = subprocess.run(['conda', 'env', 'list'], capture_output=True, text=True)
    if 'dynamicbind-minimal' in result.stdout:
        print("✓ dynamicbind-minimal environment found")
    else:
        print("✗ dynamicbind-minimal environment NOT found")

# Check input directory
print("\n2. Checking input directory...")
if os.path.exists('input'):
    files = os.listdir('input')
    print(f"Files in input/: {files}")
else:
    print("✗ input/ directory not found")

# Check CSV file
print("\n3. Checking data/input_compounds.csv...")
if os.path.exists('data/input_compounds.csv'):
    print("✓ CSV file exists")
    with open('data/input_compounds.csv', 'r') as f:
        print(f.read())
else:
    print("✗ CSV file not found")

# Check workdir
print("\n4. Checking workdir (model checkpoints)...")
if os.path.exists('workdir/big_score_model_sanyueqi_with_time'):
    print("✓ Model checkpoint found")
else:
    print("✗ Model checkpoint not found")
    if os.path.exists('workdir'):
        print(f"Contents of workdir/: {os.listdir('workdir')}")

# List all directories
print("\n5. Current directory structure:")
result = subprocess.run(['ls', '-la', '/content/DynamicBind'], capture_output=True, text=True)
print(result.stdout)

=== Diagnostic Check ===

1. Checking Conda environment...
✓ Running in Colab - conda not available (expected)

2. Checking input directory...
Files in input/: ['Conformer3D_COMPOUND_CID_977.sdf', 'unnamed.pdb']

3. Checking data/input_compounds.csv...
✓ CSV file exists
ligand,protein_path
/content/DynamicBind/input/Conformer3D_COMPOUND_CID_977.sdf,/content/DynamicBind/input/unnamed.pdb


4. Checking workdir (model checkpoints)...
✓ Model checkpoint found

5. Current directory structure:
total 448088
drwxr-xr-x 13 root root      4096 Dec 24 09:03 .
drwxr-xr-x  1 root root      4096 Dec 24 07:36 ..
-rw-r--r--  1 root root     24164 Dec 24 07:36 analysis.py
-rw-r--r--  1 root root      5662 Dec 24 07:36 check_structure_violations.py
-rw-r--r--  1 root root      5066 Dec 24 07:36 clean_pdb.py
-rw-r--r--  1 root root     11564 Dec 24 07:36 compute_lddt.py
drwxr-xr-x  2 root root      4096 Dec 24 07:36 confidence
drwxr-xr-x  4 root root      4096 Dec 24 09:03 data
drwxr-xr-x  3 root root   

## Step 8: Check Results

In [ ]:
import os
import subprocess
import glob

os.chdir('/content/DynamicBind')

# List results
results_dir = 'results/colab_run'
if os.path.exists(results_dir):
    print(f"✓ Results directory found: {results_dir}\n")
    result = subprocess.run(['find', results_dir, '-type', 'f'], capture_output=True, text=True)
    print("Files in results directory:")
    print(result.stdout)

    # Show summary
    print("\nGenerated files summary:")
    !du -sh results/colab_run/* 2>/dev/null | head -20
else:
    print(f"✗ Results directory not found: {results_dir}")

    # Check for logs
    print("\nSearching for log files...")
    log_files = glob.glob('results/**/run.log', recursive=True) + \
                glob.glob('results/**/inference.log', recursive=True) + \
                glob.glob('*.log')

    if log_files:
        print(f"Found {len(log_files)} log file(s):")
        for log_file in log_files[:5]:  # Show first 5
            print(f"\n--- {log_file} ---")
            with open(log_file, 'r') as f:
                lines = f.readlines()
                print(''.join(lines[-50:]))  # Last 50 lines
    else:
        print("No log files found")

    print("\nAvailable directories:")
    !ls -la results/ 2>/dev/null || echo "No results directory"


✓ Results directory found: results/colab_run

Files in results directory:
results/colab_run/complete_affinity_prediction.csv
results/colab_run/index0_idx_0/rank8_receptor_lddt0.54_affinity3.25.pdb
results/colab_run/index0_idx_0/rank5_receptor_lddt0.65_affinity3.00.pdb
results/colab_run/index0_idx_0/rank2_reverseprocess_data_list.pkl
results/colab_run/index0_idx_0/rank4_receptor_lddt0.64_affinity3.44.pdb
results/colab_run/index0_idx_0/rank9_receptor_lddt0.50_affinity3.54.pdb
results/colab_run/index0_idx_0/rank8_ligand_lddt0.54_affinity3.25.sdf
results/colab_run/index0_idx_0/rank9_reverseprocess_data_list.pkl
results/colab_run/index0_idx_0/rank1_ligand_lddt0.70_affinity2.48.sdf
results/colab_run/index0_idx_0/rank3_receptor_lddt0.64_affinity2.94.pdb
results/colab_run/index0_idx_0/rank10_reverseprocess_data_list.pkl
results/colab_run/index0_idx_0/rank7_receptor_lddt0.56_affinity3.48.pdb
results/colab_run/index0_idx_0/Conformer3D_COMPOUND_CID_977.sdf
results/colab_run/index0_idx_0/rank2_rec

## Step 9: Download Results

In [ ]:
from google.colab import files
import shutil
import os

os.chdir('/content/DynamicBind')

results_dir = 'results/colab_run'
if os.path.exists(results_dir):
    # Create a ZIP of results
    print(f"Preparing results for download...")
    shutil.make_archive('DynamicBind_results', 'zip', results_dir)

    # Download
    print(f"Downloading results...")
    files.download('DynamicBind_results.zip')
    print("\n✓ Download complete! Check your Downloads folder.")
else:
    print(f"ERROR: Results directory not found. Check inference output above for errors.")

Preparing results for download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ Download complete! Check your Downloads folder.


## Summary

You have completed DynamicBind inference on Google Colab!

**Output files include:**
- Ranked poses (`.sdf` and `.pdb` files) for the ligand-protein complex
- Affinity predictions (`.csv`)
- Intermediate data for visualization (`.pkl` files)

**Next steps:**
1. Download the ZIP file with all results
2. Extract and analyze the predicted complexes in your preferred molecular viewer (e.g., PyMOL, Chimera)
3. Evaluate binding affinities and pose quality

For more details, see: [DynamicBind Paper](https://www.nature.com/articles/s41467-024-45461-2)